In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_baselinegp as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
niter = 1000
error_samples = 100
gp_samples = 1000
kernel = "gaussian"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39863 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             error_samples = error_samples, gp_samples = gp_samples,
                             niter = niter, kernel = kernel)
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-fd177b85f11ecebceb044a512caf2808>,
 <Future: pending, key: main-dabac6dc6d8b7e2d4523e2d68733f66f>,
 <Future: pending, key: main-5588a1c18fbce0828982c4a20f2c9190>,
 <Future: pending, key: main-f532fb7d8e73d1a24f9ef61817733760>,
 <Future: pending, key: main-e2504ebbfc044047c42dc866c5bdcbd3>,
 <Future: pending, key: main-29b0de156d4c09a57a93e495abd55968>,
 <Future: pending, key: main-98f118e91a976013f0fdf0caaab3192f>,
 <Future: pending, key: main-f6b9298249bc8d6ec212884cf0572889>,
 <Future: pending, key: main-ad6af75675b718b206abb574d9afc819>,
 <Future: pending, key: main-d2e837d18d653310a54fba791632bfef>,
 <Future: pending, key: main-6e2c283b8eca1820c0084c51374a9928>,
 <Future: pending, key: main-96753bf3c34f1af6c1dc2e3fd03b8843>,
 <Future: pending, key: main-dd3e5af737172399b1f19702ec972a8f>,
 <Future: pending, key: main-46d9d8302f53fc660ddeff588e903196>,
 <Future: pending, key: main-5e83aa2fd4d6056bff6193b294bbcdd4>,
 <Future: pending, key: main-418e590af91

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_baselinegp_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}.pt".format(ntrial,n,d,noise,kernel))